# Prepare Model

In [68]:
import pickle
from utils.data_utils import batch_inference_with_vlm
from unsloth import FastVisionModel
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-3B-Instruct-bnb-4bit",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)
FastVisionModel.for_inference(model)



==((====))==  Unsloth 2025.3.19: Fast Qwen2_5_Vl patching. Transformers: 4.51.3.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 1. Max memory: 47.298 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [114]:

# load pickle file
with open("batch_data.pkl", "rb") as f:
    batch_data:dict = pickle.load(f) # type: dict

messages = batch_data["messages"]
images = batch_data["rgb_images"]
messages = [ [m] for m in messages]
images = [ [i] for i in images]
messages[1]

[{'role': 'user',
  'content': [{'type': 'image',
    'image': <PIL.Image.Image image mode=RGB size=640x480>},
   {'type': 'text',
    'text': '基于机器人的本体感知数据:[-0.60553426 -0.7151419   0.7958192   0.6989793   0.69278485 -0.08823238\n  0.7962648   0.9906394   0.9695734  -0.7103372   0.9473687   0.7211443\n  0.9960999  -0.6049482   0.1365048  -0.24480082  0.70386153  0.32014462\n  0.5234661   0.21508121  0.7437057   0.06619466  0.20449932 -0.62042344\n  0.75423676  0.03368961  0.04467025 -1.          0.2252341   0.35909453\n  0.9739318   0.98581064  0.9659812  -0.46111378  0.99783456  0.9743047\n  0.9333012  -0.2268411  -0.16786137  0.25861236  0.887341   -0.06577381\n  0.45283127 -0.31233788  0.8008731   0.10091418  0.08861702  0.4467847\n  0.8845037   0.01266375  0.00312535 -1.          0.31995556  0.02751317\n -0.20706016 -0.17886077  0.2573864   0.9783282   0.9838744   0.9663086\n -0.4117421   0.7287176   0.33601737  1.2706189   0.3597157   0.50089514]，确定下一步行动。'}]}]

# Batch Inference

In [115]:
# apply chat template
tokenizer.padding_side = "left"
input_text = tokenizer.apply_chat_template(messages,add_generation_prompt=True)
inputs = tokenizer(
    images,
    input_text,
    add_special_tokens = True,
    return_tensors = "pt",
    padding = True,
    truncation = True,
).to("cuda")
inputs["input_ids"].shape

torch.Size([2, 1416])

In [116]:
# inference
results = model.generate(**inputs,max_new_tokens = 128,use_cache=True,temperature=1.3,min_p=0.1)

In [117]:
outputs = tokenizer.batch_decode(results, skip_special_tokens=True)
print(len(outputs))
for o in outputs:
    print(o)
    print("*"*100)


2
system
You are a helpful assistant.
user
基于机器人的本体感知数据:[ 2.7009381e-02  9.4238244e-02  9.9963516e-01  9.9554968e-01
 -4.5955130e-01 -1.7407253e-01  4.9118367e-01  1.0264358e-01
  5.5968635e-02 -5.1123899e-01  4.0019637e-01  8.8815123e-01
  9.8473281e-01  8.7105602e-01  9.9471819e-01  9.9843252e-01
  8.5943860e-01  9.1642940e-01 -3.6645555e-01  1.0376803e+00
  8.0084181e-01  5.4243791e-01  5.1614511e-01  4.5763934e-01
  4.7950140e-01  1.2048919e-03  5.7669193e-03 -1.0000000e+00
  4.2075670e-01  3.9120566e-02  3.3916235e-01  3.3890128e-02
 -4.9243736e-01 -1.3303520e-02 -4.6921518e-01  9.0717351e-01
  9.9923450e-01  9.4072789e-01  9.9942559e-01  8.7034786e-01
  9.9991149e-01  8.8308388e-01  3.4915221e-01 -1.0533147e+00
  6.4651626e-01  8.4246987e-01 -5.2717876e-01  5.1323596e-02
 -9.8453596e-02  1.6180888e-02  7.2644786e-03 -1.0000000e+00
  1.8311219e-04 -2.3660088e-04  1.3373031e-01  1.2152614e-01
  5.1584029e-01  9.9101776e-01  9.9258822e-01 -8.5668474e-01
  9.0751439e-02  1.8374002e-0